# Bike Sharing in Washington D.C.

Paul Jacques-Mignault | May 2019  

***

## Introduction

### Objectives

This case study of the Washington D.C Bike Sharing System aims to predict the total number of users on an hourly basis. The dataset is [available on Kaggle](https://www.kaggle.com/marklvl/bike-sharing-dataset/home). It contains usage information of years 2011 and 2012. The objective is to use Dask data structures and ecosystem instead of plain pandas.

All the files of this project are saved in a [GitHub repository](https://github.com/ashomah/Bike-Sharing-in-Washington-DC). Note the csv files are stored in a different repositories than the present script. 

### Libraries

This project uses a set of libraries for data manipulation, ploting and modelling.

In [1]:
from distributed import Client, progress

In [2]:
client = Client()

In [3]:
client

<Client: scheduler='tcp://127.0.0.1:52510' processes=8 cores=8>

In [4]:
# Loading Libraries
# Timeseries CV - version 0.20.1
from sklearn.model_selection import TimeSeriesSplit, cross_validate
# Feature Selection - version 0.20.1
from sklearn.feature_selection import RFE, VarianceThreshold
from sklearn.metrics import accuracy_score  # Metrics - version 0.20.1
from sklearn.metrics import mean_squared_error, r2_score  # Metrics - version 0.20.1
from sklearn.linear_model import LinearRegression  # Model - version 0.20.1
from sklearn import datasets, linear_model  # Model - version 0.20.1

import statsmodels.api as sm  # Outliers detection - version 0.9.0
from scipy.special import boxcox1p  # Preprocessing - version 1.1.0
from scipy.stats import skew, boxcox_normmax  # Preprocessing - version 1.1.0

from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.ensemble import RandomForestRegressor
from sklearn.externals.joblib import parallel_backend

import dask.dataframe as dd  # Data Manipulation - version 1.2.2
import dask_ml.preprocessing  # Preprocessing - version 0.12.0
import dask_ml.linear_model  # Preprocessing - version 0.12.0
import dask.array as da  # Data Manipulation - version 1.2.2
from dask_ml.datasets import make_regression  # Preprocessing - version 0.12.0
# Model Selection - version 0.12.0
from dask_ml.model_selection import train_test_split

import numpy as np  # Data Manipulation - version 1.15.4
import datetime

### Data Loading

The dataset is stored in the [GitHub repository](https://github.com/ashomah/Bike-Sharing-in-Washington-DC) consisting in two CSV file: `day.csv` and `hour.csv`. The files are loaded directly from the repository. Note the csv files are stored in a different repositories than the present script. 

In [5]:
hours_df = dd.read_csv("https://raw.githubusercontent.com/ashomah/Bike-Sharing-in-Washington-DC/master/assets/data/hour.csv", date_parser = ["dteday"], blocksize = None)
hours_df.head()

instant      dteday  season   yr  mnth   hr  holiday  weekday  workingday  \
0        1  2011-01-01     1.0  0.0   1.0  0.0      0.0      6.0         0.0   
1        2  2011-01-01     1.0  0.0   1.0  1.0      0.0      6.0         0.0   
2        3  2011-01-01     1.0  0.0   1.0  2.0      0.0      6.0         0.0   
3        4  2011-01-01     1.0  0.0   1.0  3.0      0.0      6.0         0.0   
4        5  2011-01-01     1.0  0.0   1.0  4.0      0.0      6.0         0.0   

   weathersit  temp   atemp   hum  windspeed  casual  registered   cnt  
0         1.0  0.24  0.2879  0.81        0.0     3.0        13.0  16.0  
1         1.0  0.22  0.2727  0.80        0.0     8.0        32.0  40.0  
2         1.0  0.22  0.2727  0.80        0.0     5.0        27.0  32.0  
3         1.0  0.24  0.2879  0.75        0.0     3.0        10.0  13.0  
4         1.0  0.24  0.2879  0.75        0.0     0.0         1.0   1.0

In [6]:
days_df = dd.read_csv("https://raw.githubusercontent.com/ashomah/Bike-Sharing-in-Washington-DC/master/assets/data/day.csv", parse_dates = ["dteday"], blocksize = None)
days_df.head()

instant     dteday  season  yr  mnth  holiday  weekday  workingday  \
0        1 2011-01-01       1   0     1        0        6           0   
1        2 2011-01-02       1   0     1        0        0           0   
2        3 2011-01-03       1   0     1        0        1           1   
3        4 2011-01-04       1   0     1        0        2           1   
4        5 2011-01-05       1   0     1        0        3           1   

   weathersit      temp     atemp       hum  windspeed  casual  registered  \
0           2  0.344167  0.363625  0.805833   0.160446     331         654   
1           2  0.363478  0.353739  0.696087   0.248539     131         670   
2           1  0.196364  0.189405  0.437273   0.248309     120        1229   
3           1  0.200000  0.212122  0.590435   0.160296     108        1454   
4           1  0.226957  0.229270  0.436957   0.186900      82        1518   

      cnt  
0   985.0  
1   801.0  
2  1349.0  
3  1562.0  
4  1600.0

In [7]:
# Shape of the data frame
print('{:<9} {:>6} {:>6} {:>3} {:>6}'.format('hour.csv:', len(hours_df),'rows |', len(hours_df.columns), 'columns'))
print('{:<9} {:>6} {:>6} {:>3} {:>6}'.format('day.csv:', len(days_df),'rows |', len(days_df.columns), 'columns'))

hour.csv:  17379 rows |  17 columns
day.csv:     731 rows |  16 columns


## Data Preparation

### Variables Types and Definitions

The first stage of this analysis is to describe the dataset, understand the meaning of variable and perform the necessary adjustments to ensure that the data will be proceeded correctly during the Machine Learning process.

In [8]:
# Describe each variable
def df_desc(df):
    import dask.dataframe as dd
    desc = dd.DataFrame({'dtype': df.dtypes,
                         'NAs': df.isna().sum(),
                         'Numerical': (df.dtypes != 'object') & (df.dtypes != 'datetime64[ns]') & (df.apply(lambda column: column == 0).sum() + df.apply(lambda column: column == 1).sum() != len(df)),
                         'Boolean': df.apply(lambda column: column == 0).sum() + df.apply(lambda column: column == 1).sum() == len(df),
                         'Categorical': df.dtypes == 'object',
                         'Date': df.dtypes == 'datetime64[ns]',
                         })
    return desc

The dataset `day.csv` consists in 731 rows and 16 columns. The dataset `hour.csv` consists in 17,379 rows and 17 columns. Both datasets have the same columns, with an additional column for hours in `hour.csv`.

Each row provides information for each day or each hour. None of the attributes contains any NA. Four (4) of these attributes contain decimal numbers, nine (9) contain integers, three (3) contain booleans, and one (1) contains date values stored as string.

For better readability, the columns of both data frames are renamed and data types are adjusted.

In [9]:
# HOURS DATASET
# Renaming columns names to more readable names
hours_df = hours_df.rename(columns={'instant':'id',
                        'dteday':'date',
                        'weathersit':'weather_condition',
                        'hum':'humidity',
                        'mnth':'month',
                        'cnt':'total_bikes',
                        'hr':'hour',
                        'yr':'year',
                        'temp':'actual_temp',
                        'atemp':'feeling_temp'})

# Date time conversion
hours_df.date = dd.to_datetime(hours_df.date, format='%Y-%m-%d')
    
hours_df = hours_df.categorize(columns=['season',
                                        'holiday',
                                        'weekday',
                                        'workingday',
                                        'weather_condition',
                                        'month',
                                        'year',
                                        'hour'
                                       ])
    
# DAYS DATASET
# Renaming columns names to more readable names
days_df = days_df.rename(columns={'instant':'id',
                        'dteday':'date',
                        'weathersit':'weather_condition',
                        'hum':'humidity',
                        'mnth':'month',
                        'cnt':'total_bikes',
                        'yr':'year',
                        'temp':'actual_temp',
                        'atemp':'feeling_temp'})

# Date time conversion
days_df.date = dd.to_datetime(days_df.date, format='%Y-%m-%d')
    
hours_df = hours_df.categorize(columns=['season',
                                        'holiday',
                                        'weekday',
                                        'workingday',
                                        'weather_condition',
                                        'month',
                                        'year'
                                       ])

In [10]:
hours_df.head()

id       date season year month hour holiday weekday workingday  \
0   1 2011-01-01    1.0  0.0   1.0  0.0     0.0     6.0        0.0   
1   2 2011-01-01    1.0  0.0   1.0  1.0     0.0     6.0        0.0   
2   3 2011-01-01    1.0  0.0   1.0  2.0     0.0     6.0        0.0   
3   4 2011-01-01    1.0  0.0   1.0  3.0     0.0     6.0        0.0   
4   5 2011-01-01    1.0  0.0   1.0  4.0     0.0     6.0        0.0   

  weather_condition  actual_temp  feeling_temp  humidity  windspeed  casual  \
0               1.0         0.24        0.2879      0.81        0.0     3.0   
1               1.0         0.22        0.2727      0.80        0.0     8.0   
2               1.0         0.22        0.2727      0.80        0.0     5.0   
3               1.0         0.24        0.2879      0.75        0.0     3.0   
4               1.0         0.24        0.2879      0.75        0.0     0.0   

   registered  total_bikes  
0        13.0         16.0  
1        32.0         40.0  
2        27.0         32.0  
3        10.0         13.0  
4         1.0          1.0

In [11]:
hours_df.describe()

Dask DataFrame Structure:
                    id actual_temp feeling_temp humidity windspeed   casual registered total_bikes
npartitions=1                                                                                     
               float64     float64      float64  float64   float64  float64    float64     float64
                   ...         ...          ...      ...       ...      ...        ...         ...
Dask Name: describe, 59 tasks

In [12]:
# Lists values of categorical variables
categories = {'season': hours_df['season'].unique().compute().tolist(),
              'year':hours_df['year'].unique().compute().tolist(),
              'month':hours_df['month'].unique().compute().tolist(),
              'hour':hours_df['hour'].unique().compute().tolist(),
              'holiday':hours_df['holiday'].unique().compute().tolist(),
              'weekday':hours_df['weekday'].unique().compute().tolist(),
              'workingday':hours_df['workingday'].unique().compute().tolist(),
              'weather_condition':hours_df['weather_condition'].unique().compute().tolist(),
             }
for i in sorted(categories.keys()):
    print(i+":")
    print(categories[i])
    if i != sorted(categories.keys())[-1] :print()

holiday:
[0.0, 1.0]

hour:
[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0]

month:
[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0]

season:
[1.0, 2.0, 3.0, 4.0]

weather_condition:
[1.0, 2.0, 3.0, 4.0]

weekday:
[6.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0]

workingday:
[0.0, 1.0]

year:
[0.0, 1.0]


In [13]:
days_df.head()

id       date  season  year  month  holiday  weekday  workingday  \
0   1 2011-01-01       1     0      1        0        6           0   
1   2 2011-01-02       1     0      1        0        0           0   
2   3 2011-01-03       1     0      1        0        1           1   
3   4 2011-01-04       1     0      1        0        2           1   
4   5 2011-01-05       1     0      1        0        3           1   

   weather_condition  actual_temp  feeling_temp  humidity  windspeed  casual  \
0                  2     0.344167      0.363625  0.805833   0.160446     331   
1                  2     0.363478      0.353739  0.696087   0.248539     131   
2                  1     0.196364      0.189405  0.437273   0.248309     120   
3                  1     0.200000      0.212122  0.590435   0.160296     108   
4                  1     0.226957      0.229270  0.436957   0.186900      82   

   registered  total_bikes  
0         654        985.0  
1         670        801.0  
2        1229       1349.0  
3        1454       1562.0  
4        1518       1600.0

In [14]:
days_df.describe()

Dask DataFrame Structure:
                    id   season     year    month  holiday  weekday workingday weather_condition actual_temp feeling_temp humidity windspeed   casual registered total_bikes
npartitions=1                                                                                                                                                               
               float64  float64  float64  float64  float64  float64    float64           float64     float64      float64  float64   float64  float64    float64     float64
                   ...      ...      ...      ...      ...      ...        ...               ...         ...          ...      ...       ...      ...        ...         ...
Dask Name: describe, 86 tasks

In [15]:
# Lists values of categorical variables
categories = {'season': days_df['season'].unique().compute().tolist(),
              'year':days_df['year'].unique().compute().tolist(),
              'month':days_df['month'].unique().compute().tolist(),
              'holiday':days_df['holiday'].unique().compute().tolist(),
              'weekday':days_df['weekday'].unique().compute().tolist(),
              'workingday':days_df['workingday'].unique().compute().tolist(),
              'weather_condition':days_df['weather_condition'].unique().compute().tolist(),
             }
for i in sorted(categories.keys()):
    print(i+":")
    print(categories[i])
    if i != sorted(categories.keys())[-1] :print()

holiday:
[0, 1]

month:
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

season:
[1, 2, 3, 4]

weather_condition:
[2, 1, 3]

weekday:
[6, 0, 1, 2, 3, 4, 5]

workingday:
[0, 1]

year:
[0, 1]


For this study, we will only work with the dataset `hours`. The datasets contain 17 variables with no NAs:

- `id`: numerical, integer values.  
  *Record index. __This variable won't be considered in the study.__*
  
  
- `date`: numerical, date values.  
  *Date.*


- `season`: encoded categorical, integer between 1 and 4.  
  *Season: 2=Spring, 3=Summer, 4=Fall, 1=Winter.*  
  *__Note: the seasons mentioned on the Kaggle page didn't correspond to the real seasons. We readjusted the parameters accordingly.__*


- `year`: encoded categorical, integer between 0 and 1.  
  *Year: 0=2011, 1=2012.*
  
  
- `month`: encoded categorical, integer between 1 and 12.  
  *Month.*
  
  
- `hour`: encoded categorical, integer between 1 and 23.  
  *Hour.*
  
  
- `holiday`: encoded categorical, boolean.  
  *Flag indicating if the day is a holiday.*


- `weekday`: encoded categorical, integer between 0 and 6.  
  *Day of the week (0=Sunday, ... 6=Saturday).*


- `workingday`: encoded categorical, boolean.  
  *Flag indicating if the day is a working day.*
  
  
- `weather_condition`: encoded categorical, integer between 1 and 4.  
  *Weather condition (1=Clear, 2=Mist, 3=Light Rain, 4=Heavy Rain).*


- `actual_temp`: numerical, decimal values between 0 and 1.  
  *Normalized temperature in Celsius (min = -16, max = +50).*


- `feeling_temp`: numerical, decimal values between 0 and 1.  
  *Normalized feeling temperature in Celsius (min = -8, max = +39).*


- `humidity`: numerical, decimal values between 0 and 1.  
  *Normalized humidity.*


- `windspeed`: numerical, decimal values between 0 and 1.  
  *Normalized wind speed.*


- `casual`: numerical, integer.  
  *Count of casual users. This variable won't be considered in the study.*


- `registered`: numerical, integer.  
  *Count of registered users. This variable won't be considered in the study.*


- `total_bikes`: numerical, integer.  
  *Count of total rental bikes (casual+registered). This is the __target variable__ of the study, the one to be modelled.*

In [16]:
# Remove variable id
hours_df= hours_df.drop(['id'], axis=1)

### Scaling

In [17]:
hours_prep_scaled = hours_df.copy().drop(['date','casual', 'registered', 'holiday','feeling_temp'],axis=1)

In [18]:
hours_prep_scaled.describe()

Dask DataFrame Structure:
              actual_temp humidity windspeed total_bikes
npartitions=1                                           
                  float64  float64   float64     float64
                      ...      ...       ...         ...
Dask Name: describe, 45 tasks

In [19]:
scaler = dask_ml.preprocessing.MinMaxScaler()
transformer = scaler.fit(hours_prep_scaled[['actual_temp', 'humidity', 'windspeed', 'total_bikes']])
hours_prep_scaled[['actual_temp', 'humidity', 'windspeed', 'total_bikes']] = transformer.transform(hours_prep_scaled[['actual_temp', 'humidity', 'windspeed', 'total_bikes']])

The scale of the dataset is corrected.

### Encoding Categorical Variables

In [20]:
hours_prep_skew = hours_prep_scaled.copy()

In [21]:
hours_prep_encoded = hours_prep_skew.copy()

In [22]:
def date_features(df):
    columns = df.columns
    return df.select_dtypes(include=[np.datetime64]).columns

def numerical_features(df):
    columns = df.columns
    return df._get_numeric_data().columns

def categorical_features(df):
    numerical_columns = numerical_features(df)
    date_columns = date_features(df)
    return(list(set(df.columns) - set(numerical_columns) - set(date_columns) ))

def onehot_encode(df):
    df_num = df[numerical_features(df)]
    df_cat = df[categorical_features(df)]
    enc = dask_ml.preprocessing.OneHotEncoder(sparse=False)
    enc.fit(df_cat)
    df_cat_enc = enc.transform(df_cat)
    
    new_df = dd.concat([df_cat_enc, df_num], axis=1)
    return new_df

def onehot_encode_single(df, variable):
    df = df.categorize(columns=variable)
    df_variable = df[variable].to_frame()
    enc = dask_ml.preprocessing.OneHotEncoder(sparse=False)
    enc.fit(df_variable)
    df_variable = enc.transform(df_variable)
    
    new_df = dd.concat([df, df_variable], axis=1)
    return new_df

In [23]:
hours_clean = onehot_encode(hours_prep_encoded)

/Users/paul.jacques-mignault/anaconda3/lib/python3.7/site-packages/dask/dataframe/multi.py:604: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indexes of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn("Concatenating dataframes with unknown divisions.\n"


In [24]:
hours_clean.head()

year_0.0  year_1.0  workingday_0.0  workingday_1.0  weather_condition_1.0  \
0       1.0       0.0             1.0             0.0                    1.0   
1       1.0       0.0             1.0             0.0                    1.0   
2       1.0       0.0             1.0             0.0                    1.0   
3       1.0       0.0             1.0             0.0                    1.0   
4       1.0       0.0             1.0             0.0                    1.0   

   weather_condition_2.0  weather_condition_3.0  weather_condition_4.0  \
0                    0.0                    0.0                    0.0   
1                    0.0                    0.0                    0.0   
2                    0.0                    0.0                    0.0   
3                    0.0                    0.0                    0.0   
4                    0.0                    0.0                    0.0   

   season_1.0  season_2.0  ...  hour_18.0  hour_19.0  hour_20.0  hour_21.0  \
0         1.0         0.0  ...        0.0        0.0        0.0        0.0   
1         1.0         0.0  ...        0.0        0.0        0.0        0.0   
2         1.0         0.0  ...        0.0        0.0        0.0        0.0   
3         1.0         0.0  ...        0.0        0.0        0.0        0.0   
4         1.0         0.0  ...        0.0        0.0        0.0        0.0   

   hour_22.0  hour_23.0  actual_temp  humidity  windspeed  total_bikes  
0        0.0        0.0     0.224490      0.81        0.0     0.015369  
1        0.0        0.0     0.204082      0.80        0.0     0.039959  
2        0.0        0.0     0.204082      0.80        0.0     0.031762  
3        0.0        0.0     0.224490      0.75        0.0     0.012295  
4        0.0        0.0     0.224490      0.75        0.0     0.000000  

[5 rows x 59 columns]

In [25]:
# Rename columns
hours_clean = hours_clean.rename(columns={'year_0.0':'year_2011',
                            'year_1.0':'year_2012',
                            'season_1.0':'season_winter',
                            'season_2.0':'season_spring',
                            'season_3.0':'season_summer',
                            'season_4.0':'season_fall',
                            'workingday_0.0':'workingday_no',
                            'workingday_1.0':'workingday_yes',
                            'month_1.0':'month_jan',
                            'month_2.0':'month_feb',
                            'month_3.0':'month_mar',
                            'month_4.0':'month_apr',
                            'month_5.0':'month_may',
                            'month_6.0':'month_jun',
                            'month_7.0':'month_jul',
                            'month_8.0':'month_aug',
                            'month_9.0':'month_sep',
                            'month_10.0':'month_oct',
                            'month_11.0':'month_nov',
                            'month_12.0':'month_dec',
                            'weather_condition_1.0':'weather_condition_clear',
                            'weather_condition_2.0':'weather_condition_mist',
                            'weather_condition_3.0':'weather_condition_light_rain',
                            'weather_condition_4.0':'weather_condition_heavy_rain',
                            'weekday_0.0':'weekday_sunday',
                            'weekday_1.0':'weekday_monday',
                            'weekday_2.0':'weekday_tuesday',
                            'weekday_3.0':'weekday_wednesday',
                            'weekday_4.0':'weekday_thursday',
                            'weekday_5.0':'weekday_friday',
                            'weekday_6.0':'weekday_saturday'})

In [26]:
hours_clean = hours_clean.drop(['workingday_no'], axis=1)

The variable `workingday_no` has been removed, as complementary of `workingday_yes`.

In [27]:
hours_clean.head()

year_2011  year_2012  workingday_yes  weather_condition_clear  \
0        1.0        0.0             0.0                      1.0   
1        1.0        0.0             0.0                      1.0   
2        1.0        0.0             0.0                      1.0   
3        1.0        0.0             0.0                      1.0   
4        1.0        0.0             0.0                      1.0   

   weather_condition_mist  weather_condition_light_rain  \
0                     0.0                           0.0   
1                     0.0                           0.0   
2                     0.0                           0.0   
3                     0.0                           0.0   
4                     0.0                           0.0   

   weather_condition_heavy_rain  season_winter  season_spring  season_summer  \
0                           0.0            1.0            0.0            0.0   
1                           0.0            1.0            0.0            0.0   
2                           0.0            1.0            0.0            0.0   
3                           0.0            1.0            0.0            0.0   
4                           0.0            1.0            0.0            0.0   

   ...  hour_18.0  hour_19.0  hour_20.0  hour_21.0  hour_22.0  hour_23.0  \
0  ...        0.0        0.0        0.0        0.0        0.0        0.0   
1  ...        0.0        0.0        0.0        0.0        0.0        0.0   
2  ...        0.0        0.0        0.0        0.0        0.0        0.0   
3  ...        0.0        0.0        0.0        0.0        0.0        0.0   
4  ...        0.0        0.0        0.0        0.0        0.0        0.0   

   actual_temp  humidity  windspeed  total_bikes  
0     0.224490      0.81        0.0     0.015369  
1     0.204082      0.80        0.0     0.039959  
2     0.204082      0.80        0.0     0.031762  
3     0.224490      0.75        0.0     0.012295  
4     0.224490      0.75        0.0     0.000000  

[5 rows x 58 columns]

The dataset is ready for modeling.

### Training/Test Split

In [28]:
def list_features(df, target):
    features = list(df.columns)
    features.remove(target)
    return features

In [29]:
target = 'total_bikes'
features = list_features(hours_clean, target)

In [30]:
X = hours_clean[features]
X_train = X.loc[(X['year_2011']==1) | ((X['year_2012']==1) & (X['month_sep']==0) & (X['month_oct']==0) & (X['month_nov']==0) & (X['month_dec']==0)),features]
X_test = X.loc[(X['year_2012']==1) & ((X['month_sep']==1) | (X['month_oct']==1) | (X['month_nov']==1) | (X['month_dec']==1)),features]
print('{:<9} {:>6} {:>6} {:>3} {:>6}'.format('X_train:', len(X_train),'rows |', len(X_train.columns), 'columns'))
print('{:<9} {:>6} {:>6} {:>3} {:>6}'.format('X_test:', len(X_test),'rows |', len(X_test.columns), 'columns'))

X_train:   14491 rows |  57 columns
X_test:     2888 rows |  57 columns


In [31]:
X_train.groupby(['year_2011','year_2012','month_jan','month_feb','month_mar','month_apr','month_may','month_jun','month_jul','month_aug','month_sep','month_oct','month_nov','month_dec']).size().reset_index()

Dask DataFrame Structure:
              year_2011 year_2012 month_jan month_feb month_mar month_apr month_may month_jun month_jul month_aug month_sep month_oct month_nov month_dec      0
npartitions=1                                                                                                                                                   
                float64   float64   float64   float64   float64   float64   float64   float64   float64   float64   float64   float64   float64   float64  int64
                    ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...    ...
Dask Name: reset_index, 47 tasks

In [32]:
X_test.groupby(['year_2011','year_2012','month_jan','month_feb','month_mar','month_apr','month_may','month_jun','month_jul','month_aug','month_sep','month_oct','month_nov','month_dec']).size().reset_index()

Dask DataFrame Structure:
              year_2011 year_2012 month_jan month_feb month_mar month_apr month_may month_jun month_jul month_aug month_sep month_oct month_nov month_dec      0
npartitions=1                                                                                                                                                   
                float64   float64   float64   float64   float64   float64   float64   float64   float64   float64   float64   float64   float64   float64  int64
                    ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...    ...
Dask Name: reset_index, 44 tasks

In [33]:
y = hours_clean.copy()
y_train = y.loc[(y['year_2011']==1) | ((y['year_2012']==1) & (y['month_sep']==0) & (y['month_oct']==0) & (y['month_nov']==0) & (y['month_dec']==0)),:]
y_test = y.loc[(y['year_2012']==1) & ((y['month_sep']==1) | (y['month_oct']==1) | (y['month_nov']==1) | (y['month_dec']==1)),:]
y_train = y_train[target].to_frame()
y_test = y_test[target].to_frame()
print('{:<9} {:>6} {:>6} {:>3} {:>6}'.format('y_train:', len(y_train),'rows |', len(y_train.columns), 'columns'))
print('{:<9} {:>6} {:>6} {:>3} {:>6}'.format('y_test:', len(y_test),'rows |', len(y_test.columns), 'columns'))

y_train:   14491 rows |   1 columns
y_test:     2888 rows |   1 columns


In [34]:
print('{:<35} {!r:>}'.format('Same indexes for X_train and y_train:', X_train.index.values.compute().tolist() == y_train.index.values.compute().tolist()))
print('{:<35} {!r:>}'.format('Same indexes for X_test and y_test:', X_test.index.values.compute().tolist() == y_test.index.values.compute().tolist()))

Same indexes for X_train and y_train: True
Same indexes for X_test and y_test: True


In [35]:
print('{:<15} {:>6} {:>6} {:>2} {:>6}'.format('Features:',len(X), 'items | ', len(X.columns),'columns'))
print('{:<15} {:>6} {:>6} {:>2} {:>6}'.format('Features Train:', len(X_train), 'items | ', len(X_train.columns),'columns'))
print('{:<15} {:>6} {:>6} {:>2} {:>6}'.format('Features Test:', len(X_test), 'items | ', len(X_test.columns),'columns'))
print('{:<15} {:>6} {:>6} {:>2} {:>6}'.format('Target:', len(y), 'items | ', len(y.columns),'columns'))
print('{:<15} {:>6} {:>6} {:>2} {:>6}'.format('Target Train:', len(y_train), 'items | ', len(y_train.columns),'columns'))
print('{:<15} {:>6} {:>6} {:>2} {:>6}'.format('Target Test:', len(y_test), 'items | ', len(y_test.columns),'columns'))

Features:        17379 items |  57 columns
Features Train:  14491 items |  57 columns
Features Test:    2888 items |  57 columns
Target:          17379 items |  58 columns
Target Train:    14491 items |   1 columns
Target Test:      2888 items |   1 columns


The Train Set is arbitrarily defined as all records until August 31st 2012, and the Test Set all records from September 1st 2012. Below function will be used to repeat the operation on future dataframes including new features.

In [36]:
def train_test_split_0(df, target):
    X = df.copy()
    features = list_features(df, target)
    X = df[features]    
    y = df.copy()
    X_train = X.loc[(X['year_2011']==1) | ((X['year_2012']==1) & (X['month_sep']==0) & (X['month_oct']==0) & (X['month_nov']==0) & (X['month_dec']==0)),features]
    X_test = X.loc[(X['year_2012']==1) & ((X['month_sep']==1) | (X['month_oct']==1) | (X['month_nov']==1) | (X['month_dec']==1)),features]
    y_train = y.loc[(y['year_2011']==1) | ((y['year_2012']==1) & (y['month_sep']==0) & (y['month_oct']==0) & (y['month_nov']==0) & (y['month_dec']==0)),:]
    y_test = y.loc[(y['year_2012']==1) & ((y['month_sep']==1) | (y['month_oct']==1) | (y['month_nov']==1) | (y['month_dec']==1)),:]
    y_train = y_train[target].to_frame()
    y_test = y_test[target].to_frame()
    y = df[target].to_frame()
    
    print('{:<40} {!r:>}'.format('Same indexes for X and y:', X.index.values.compute().tolist() == y.index.values.compute().tolist()))
    print('{:<40} {!r:>}'.format('Same indexes for X_train and y_train:', X_train.index.values.compute().tolist() == y_train.index.values.compute().tolist()))
    print('{:<40} {!r:>}'.format('Same indexes for X_test and y_test:', X_test.index.values.compute().tolist() == y_test.index.values.compute().tolist()))
    print()
    print('{:<15} {:>6} {:>6} {:>2} {:>6}'.format('Features:',len(X), 'items | ', len(X.columns),'columns'))
    print('{:<15} {:>6} {:>6} {:>2} {:>6}'.format('Features Train:', len(X_train), 'items | ', len(X_train.columns),'columns'))
    print('{:<15} {:>6} {:>6} {:>2} {:>6}'.format('Features Test:', len(X_test), 'items | ',  len(X_test.columns),'columns'))
    print('{:<15} {:>6} {:>6} {:>2} {:>6}'.format('Target:', len(y), 'items | ', len(y.columns),'columns'))
    print('{:<15} {:>6} {:>6} {:>2} {:>6}'.format('Target Train:', len(y_train), 'items | ', len(y_train.columns),'columns'))
    print('{:<15} {:>6} {:>6} {:>2} {:>6}'.format('Target Test:', len(y_test), 'items | ', len(y_test.columns),'columns'))
    print()
    
    return X, X_train, X_test, y, y_train, y_test

## Baseline

In [37]:
with parallel_backend('dask'):

    lm = dask_ml.linear_model.LinearRegression()
    lm.fit(X_train.values, y_train.values)
    y_pred = lm.predict(X_test.values).compute()

    print('Intercept:', lm.intercept_)
    print('Coefficients:', lm.coef_)
    print('Mean squared error (MSE): {:.2f}'.format(mean_squared_error(y_test, y_pred)))
    print('Variance score (R2): {:.2f}'.format(r2_score(y_test, y_pred)))

Intercept: -0.032772611699891416
Coefficients: [ 0.12162194 -0.04392317  0.04396376  0.01974657  0.01310665  0.00426076
 -0.04805843 -0.04981442 -0.02028516  0.00984738 -0.00362692  0.02145953
  0.01835533  0.00569505 -0.00881901 -0.0048717  -0.00493926 -0.00377265
 -0.00154845 -0.02316936 -0.01755833 -0.00141292 -0.00207618  0.01345699
  0.00368002 -0.01022297  0.00896718  0.02316777  0.01795153  0.00422598
  0.00560081 -0.12064328 -0.13783824 -0.14593734 -0.15676478 -0.15987671
 -0.14391122 -0.08705978  0.04151926  0.17492107  0.03581461 -0.01690215
  0.0060972   0.04260202  0.03871467  0.02368416  0.03150675  0.09118695
  0.24367112  0.2166576   0.11396975  0.03651263 -0.01140959 -0.04859113
 -0.08834253  0.20993633 -0.07662468]
Mean squared error (MSE): 0.02
Variance score (R2): 0.63


The baseline model for our dataset obtains a R square of 0.63 with 57 features.

## Feature Engineering

### Features Construction

#### Pipeline

Each new feature will be tested through below pipeline.

In [38]:
def pipeline(df, target): 
    
    X, X_train, X_test, y, y_train, y_test = train_test_split_0(df, target)
    lm = dask_ml.linear_model.LinearRegression()
    lm.fit(X_train.values, y_train.values)
    y_pred = lm.predict(X_test.values).compute()

    print()
    print('Intercept:', lm.intercept_)
    print('Coefficients:', lm.coef_)
    print('Mean squared error (MSE): {:.2f}'.format(mean_squared_error(y_test, y_pred)))
    print('Variance score (R2): {:.2f}'.format(r2_score(y_test, y_pred)))

#### Calculated Peaks

The variable `calculated_peak` is added to flag the periods of typically high utilization.

In [39]:
hours_FE1 = hours_clean.copy()

In [40]:
th = 0.315

def calculated_peaks(row, th):
    if (hours_FE1.loc[row, 'total_bikes'] > (hours_FE1.total_bikes.mean()+ th *hours_FE1.total_bikes.mean())):
        return 1
    else:
        return 0

hours_FE1['calculated_peak'] = hours_FE1.apply(lambda row: calculated_peaks(row, th), axis=1, meta=(None, 'float64'))
hours_FE1.calculated_peak.value_counts()

Dask Series Structure:
npartitions=1
    int64
      ...
Name: calculated_peak, dtype: int64
Dask Name: value-counts-agg, 29 tasks

In [41]:
hours_FE1.describe()

Dask DataFrame Structure:
              year_2011 year_2012 workingday_yes weather_condition_clear weather_condition_mist weather_condition_light_rain weather_condition_heavy_rain season_winter season_spring season_summer season_fall weekday_saturday weekday_sunday weekday_monday weekday_tuesday weekday_wednesday weekday_thursday weekday_friday month_jan month_feb month_mar month_apr month_may month_jun month_jul month_aug month_sep month_oct month_nov month_dec hour_0.0 hour_1.0 hour_2.0 hour_3.0 hour_4.0 hour_5.0 hour_6.0 hour_7.0 hour_8.0 hour_9.0 hour_10.0 hour_11.0 hour_12.0 hour_13.0 hour_14.0 hour_15.0 hour_16.0 hour_17.0 hour_18.0 hour_19.0 hour_20.0 hour_21.0 hour_22.0 hour_23.0 actual_temp humidity windspeed total_bikes calculated_peak
npartitions=1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
                float64   float64        float64                 float64                float64                      float64                      float64       float64       float64       float64     float64          float64        float64        float64         float64           float64          float64        float64   float64   float64   float64   float64   float64   float64   float64   float64   float64   float64   float64   float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64   float64   float64   float64   float64   float64   float64   float64   float64   float64   float64   float64   float64   float64   float64     float64  float64   float64     float64         float64
                    ...       ...            ...                     ...                    ...                          ...                          ...           ...           ...           ...         ...              ...            ...            ...             ...               ...              ...            ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...         ...      ...       ...         ...             ...
Dask Name: describe, 280 tasks

In [42]:
client.compute(hours_FE1)

<Future: status: pending, key: finalize-c97a46d7b760d84c8fe50c5c3a7324e6>

In [43]:
# pipeline(hours_FE1, 'total_bikes')

The above is a failed intent to compute hours_FE1 using the client. The computation does not seem to work. 

#### Difference with Season Average Temperature

The variable `diff_season_avg_temp` is added to identify days with clement temperature compared to the seasonal average.

In [44]:
hours_FE_sel  = hours_clean.copy()

In [45]:
hours_FE2 = hours_FE_sel.copy()

def diff_season_avg_temp_calc(df, row):
    if (df.loc[row, 'actual_temp'] == 1):
        return (df.loc[row, 'actual_temp'] - df.actual_temp[df.season_spring == 1].mean())
    
    elif (df.loc[row, 'actual_temp'] == 1):
        return (df.loc[row, 'actual_temp'] - df.actual_temp[df.season_winter == 1].mean())
    
    elif (df.loc[row, 'actual_temp'] == 1):
        return (df.loc[row, 'actual_temp'] - df.actual_temp[df.season_fall == 1].mean())
    
    elif (df.loc[row, 'actual_temp'] == 1):
        return (df.loc[row, 'actual_temp'] - df.actual_temp[df.season_summer == 1].mean())
    
hours_FE2['diff_season_avg_temp'] = hours_FE2.apply(lambda row: diff_season_avg_temp_calc(hours_FE2, row), axis=1, meta=(None, 'int64'))

In [46]:
hours_FE2.describe()

Dask DataFrame Structure:
              year_2011 year_2012 workingday_yes weather_condition_clear weather_condition_mist weather_condition_light_rain weather_condition_heavy_rain season_winter season_spring season_summer season_fall weekday_saturday weekday_sunday weekday_monday weekday_tuesday weekday_wednesday weekday_thursday weekday_friday month_jan month_feb month_mar month_apr month_may month_jun month_jul month_aug month_sep month_oct month_nov month_dec hour_0.0 hour_1.0 hour_2.0 hour_3.0 hour_4.0 hour_5.0 hour_6.0 hour_7.0 hour_8.0 hour_9.0 hour_10.0 hour_11.0 hour_12.0 hour_13.0 hour_14.0 hour_15.0 hour_16.0 hour_17.0 hour_18.0 hour_19.0 hour_20.0 hour_21.0 hour_22.0 hour_23.0 actual_temp humidity windspeed total_bikes diff_season_avg_temp
npartitions=1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
                float64   float64        float64                 float64                float64                      float64                      float64       float64       float64       float64     float64          float64        float64        float64         float64           float64          float64        float64   float64   float64   float64   float64   float64   float64   float64   float64   float64   float64   float64   float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64   float64   float64   float64   float64   float64   float64   float64   float64   float64   float64   float64   float64   float64   float64     float64  float64   float64     float64              float64
                    ...       ...            ...                     ...                    ...                          ...                          ...           ...           ...           ...         ...              ...            ...            ...             ...               ...              ...            ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...         ...      ...       ...         ...                  ...
Dask Name: describe, 280 tasks

In [47]:
client.compute(hours_FE2)

<Future: status: pending, key: finalize-eb028967ea063a9cb386d23697aa6215>

In [48]:
# pipeline(hours_FE2, 'total_bikes')

The above is a failed intent to compute hours_FE1 using the client. The computation does not seem to work. 

#### Polynomial Features

The polynomial variables of degree 2 are added to identify potential polynomial relationship between the target and the corresponding variables. The columns with a variance = 0 will be dropped from the resulting dataframe.  

In [49]:
hours_FE3 = hours_clean.copy()

In [50]:
poly = dask_ml.preprocessing.PolynomialFeatures(2)
hours_FE3 = poly.fit_transform(hours_FE3)

In [51]:
hours_FE3 = hours_FE3.to_dask_dataframe()

In [52]:
lst = list(range(len(hours_clean.columns)))
lst = [x+1 for x in lst]

values = list(hours_clean.columns)
rename_dict = dict(zip(lst, values))

In [53]:
hours_FE3 = hours_FE3.rename(columns=rename_dict)

In [54]:
constant_filter = VarianceThreshold(threshold = 0)  
constant_filter.fit(hours_FE3)

len(hours_FE3.columns[constant_filter.get_support()])

constant_columns = [column for column in hours_FE3.columns  
                    if column not in hours_FE3.columns[constant_filter.get_support()]]

hours_FE3 = hours_FE3.drop(labels=constant_columns, axis=1)  

In [55]:
hours_FE3.head()

year_2011  year_2012  workingday_yes  weather_condition_clear  \
0        1.0        0.0             0.0                      1.0   
1        1.0        0.0             0.0                      1.0   
2        1.0        0.0             0.0                      1.0   
3        1.0        0.0             0.0                      1.0   
4        1.0        0.0             0.0                      1.0   

   weather_condition_mist  weather_condition_light_rain  \
0                     0.0                           0.0   
1                     0.0                           0.0   
2                     0.0                           0.0   
3                     0.0                           0.0   
4                     0.0                           0.0   

   weather_condition_heavy_rain  season_winter  season_spring  season_summer  \
0                           0.0            1.0            0.0            0.0   
1                           0.0            1.0            0.0            0.0   
2                           0.0            1.0            0.0            0.0   
3                           0.0            1.0            0.0            0.0   
4                           0.0            1.0            0.0            0.0   

   ...      1760      1761  1762      1763    1764  1765      1766  1767  \
0  ...  0.050396  0.181837   0.0  0.003450  0.6561   0.0  0.012449   0.0   
1  ...  0.041649  0.163265   0.0  0.008155  0.6400   0.0  0.031967   0.0   
2  ...  0.041649  0.163265   0.0  0.006482  0.6400   0.0  0.025410   0.0   
3  ...  0.050396  0.168367   0.0  0.002760  0.5625   0.0  0.009221   0.0   
4  ...  0.050396  0.168367   0.0  0.000000  0.5625   0.0  0.000000   0.0   

   1768      1769  
0   0.0  0.000236  
1   0.0  0.001597  
2   0.0  0.001009  
3   0.0  0.000151  
4   0.0  0.000000  

[5 rows x 1320 columns]

### Model Selection

In [56]:
with parallel_backend('dask'):
    pipeline(hours_clean, 'total_bikes')

Same indexes for X and y:                True
Same indexes for X_train and y_train:    True
Same indexes for X_test and y_test:      True

Features:        17379 items |  57 columns
Features Train:  14491 items |  57 columns
Features Test:    2888 items |  57 columns
Target:          17379 items |   1 columns
Target Train:    14491 items |   1 columns
Target Test:      2888 items |   1 columns


Intercept: -0.032772611699891416
Coefficients: [ 0.12162194 -0.04392317  0.04396376  0.01974657  0.01310665  0.00426076
 -0.04805843 -0.04981442 -0.02028516  0.00984738 -0.00362692  0.02145953
  0.01835533  0.00569505 -0.00881901 -0.0048717  -0.00493926 -0.00377265
 -0.00154845 -0.02316936 -0.01755833 -0.00141292 -0.00207618  0.01345699
  0.00368002 -0.01022297  0.00896718  0.02316777  0.01795153  0.00422598
  0.00560081 -0.12064328 -0.13783824 -0.14593734 -0.15676478 -0.15987671
 -0.14391122 -0.08705978  0.04151926  0.17492107  0.03581461 -0.01690215
  0.0060972   0.04260202  0.03871467  0.023

In [62]:
with parallel_backend('dask'):
    X, X_train, X_test, y, y_train, y_test = train_test_split_0(hours_clean, 'total_bikes')
    regressor = RandomForestRegressor(n_estimators=20, random_state=1)  
    regressor.fit(X_train.values, y_train.values)  
    y_pred = regressor.predict(X_test)
    
    print('Mean Squared Error:', mean_squared_error(y_test, y_pred))
    print('R2 Scores:', r2_score(y_test, y_pred))  

Same indexes for X and y:                True
Same indexes for X_train and y_train:    True
Same indexes for X_test and y_test:      True

Features:        17379 items |  57 columns
Features Train:  14491 items |  57 columns
Features Test:    2888 items |  57 columns
Target:          17379 items |   1 columns
Target Train:    14491 items |   1 columns
Target Test:      2888 items |   1 columns



/Users/paul.jacques-mignault/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


Mean Squared Error: 0.008069493709044844
R2 Scores: 0.8377660077586028


In [64]:
with parallel_backend('dask'):
    X, X_train, X_test, y, y_train, y_test = train_test_split_0(hours_FE3, 'total_bikes')
    regressor = RandomForestRegressor(n_estimators=5, random_state=1, max_depth = 5, max_features = 100)  
    regressor.fit(X_train.values, y_train.values)  
    y_pred = regressor.predict(X_test)
    
    print('Mean Squared Error:', mean_squared_error(y_test, y_pred))
    print('R2 Scores:', r2_score(y_test, y_pred))  

Same indexes for X and y:                True
Same indexes for X_train and y_train:    True
Same indexes for X_test and y_test:      True

Features:        17379 items |  1319 columns
Features Train:  14491 items |  1319 columns
Features Test:    2888 items |  1319 columns
Target:          17379 items |   1 columns
Target Train:    14491 items |   1 columns
Target Test:      2888 items |   1 columns



/Users/paul.jacques-mignault/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


Mean Squared Error: 0.004832658186828465
R2 Scores: 0.9028413108608699


## Final Metric

The Feature Selection phase suggested several acceptable models, which mainly differ on the number of features used and the resulting performance. Depending on the accuracy expected by the business, one of these models will be selected:  

1. Baseline Model with Linear Regression:  
Score: 0.63 | Features: 57    


2. Random Forest Model:  
Score: 0.84 | Features: 57  


3. Random Forest Model with Polynomial of Degree 2:   
Score: 0.90 | Features: 1319  

Based on the predictions and the way they fit the actual values, model 3 is preferable. Polynomials may lead to overfitting; model 2 would present a more conservative approach with more stable but slightly worse performance.

***

*Paul Jacques-Mignault | May 2019*  

***